# Evolver Loop 1 - LB Feedback Analysis

**Baseline submitted:** CV 70.6761 | LB 70.6761 (gap: 0.0000)

This confirms our local score calculation is accurate. Now we need to improve by 1.757 points (2.55%) to beat the target of 68.919154.

In [ ]:
import pandas as pd
import numpy as np
import os

# Load the baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(baseline_path)
print(f"Loaded {len(df)} rows")
print(df.head())

In [ ]:
# Parse values and calculate per-N scores
def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_vertices(x, y, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    vx = x + TX * c - TY * s
    vy = y + TX * s + TY * c
    return vx, vy

def calculate_per_n_scores(df):
    scores = {}
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        
        all_x, all_y = [], []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            vx, vy = get_vertices(x, y, deg)
            all_x.extend(vx)
            all_y.extend(vy)
        
        side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
        score = side**2 / n
        scores[n] = {'side': side, 'score': score}
    return scores

scores = calculate_per_n_scores(df)
total = sum(s['score'] for s in scores.values())
print(f"Total score: {total:.6f}")

In [ ]:
# Analyze score distribution
import matplotlib.pyplot as plt

n_values = list(scores.keys())
score_values = [scores[n]['score'] for n in n_values]
side_values = [scores[n]['side'] for n in n_values]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Score contribution by N
axes[0].bar(n_values, score_values, alpha=0.7)
axes[0].set_xlabel('N')
axes[0].set_ylabel('Score contribution (s²/n)')
axes[0].set_title('Score Contribution by N')
axes[0].axhline(y=np.mean(score_values), color='r', linestyle='--', label=f'Mean: {np.mean(score_values):.4f}')
axes[0].legend()

# Side length by N
axes[1].scatter(n_values, side_values, alpha=0.7, s=10)
axes[1].set_xlabel('N')
axes[1].set_ylabel('Side length')
axes[1].set_title('Side Length by N')

# Add theoretical lower bound (sqrt(n * tree_area))
tree_area = 0.2456  # from earlier calculation
theoretical_min = [np.sqrt(n * tree_area) for n in n_values]
axes[1].plot(n_values, theoretical_min, 'r--', label='Theoretical min (sqrt(n*area))')
axes[1].legend()

plt.tight_layout()
plt.savefig('/home/code/exploration/score_analysis.png', dpi=100)
plt.show()

print("\nTop 20 score contributors:")
sorted_scores = sorted(scores.items(), key=lambda x: x[1]['score'], reverse=True)[:20]
for n, data in sorted_scores:
    print(f"N={n:3d}: side={data['side']:.4f}, score={data['score']:.4f}")

In [ ]:
# Calculate packing efficiency for each N
print("\nPacking efficiency analysis:")
print("(Efficiency = n * tree_area / side²)")
print("\nLowest efficiency (most room for improvement):")

efficiencies = {}
for n, data in scores.items():
    efficiency = n * tree_area / (data['side'] ** 2)
    efficiencies[n] = efficiency

sorted_eff = sorted(efficiencies.items(), key=lambda x: x[1])[:20]
for n, eff in sorted_eff:
    print(f"N={n:3d}: efficiency={eff:.4f}, side={scores[n]['side']:.4f}, score={scores[n]['score']:.4f}")

In [ ]:
# Check N=1 specifically - is 45 degrees optimal?
print("\nN=1 Analysis:")
n1_row = df[df['id'] == '001_0'].iloc[0]
x1 = parse_value(n1_row['x'])
y1 = parse_value(n1_row['y'])
deg1 = parse_value(n1_row['deg'])
print(f"Current: x={x1:.4f}, y={y1:.4f}, deg={deg1:.4f}")

# Try different angles for N=1
print("\nTrying different angles for N=1:")
best_angle = None
best_side = float('inf')
for angle in np.arange(0, 360, 0.1):
    vx, vy = get_vertices(0, 0, angle)
    side = max(max(vx) - min(vx), max(vy) - min(vy))
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Best angle: {best_angle:.1f} degrees")
print(f"Best side: {best_side:.6f}")
print(f"Best score for N=1: {best_side**2:.6f}")
print(f"Current score for N=1: {scores[1]['score']:.6f}")
print(f"Potential improvement: {scores[1]['score'] - best_side**2:.6f}")

In [ ]:
# Check what other pre-optimized files are available
import glob

preopt_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'
all_csvs = glob.glob(f"{preopt_dir}/**/*.csv", recursive=True)
print(f"Found {len(all_csvs)} CSV files")

# Calculate scores for each
file_scores = {}
for csv_path in all_csvs[:20]:  # Limit to first 20 for speed
    try:
        df_temp = pd.read_csv(csv_path)
        if 'id' in df_temp.columns and 'x' in df_temp.columns:
            temp_scores = calculate_per_n_scores(df_temp)
            total = sum(s['score'] for s in temp_scores.values())
            file_scores[csv_path] = total
    except:
        pass

print("\nFile scores (sorted):")
for path, score in sorted(file_scores.items(), key=lambda x: x[1])[:10]:
    print(f"{score:.4f}: {os.path.basename(path)}")

In [ ]:
# Summary of findings
print("="*60)
print("SUMMARY OF FINDINGS")
print("="*60)
print(f"\n1. Baseline score: 70.6761 (verified on LB)")
print(f"2. Target score: 68.9192")
print(f"3. Gap to close: 1.757 points (2.55%)")
print(f"\n4. N=1 optimization potential:")
print(f"   - Current: {scores[1]['score']:.6f}")
print(f"   - Optimal (45°): {best_side**2:.6f}")
print(f"   - Improvement: {scores[1]['score'] - best_side**2:.6f}")
print(f"\n5. Top contributors to score (small N):")
for n, data in sorted_scores[:10]:
    print(f"   N={n:3d}: {data['score']:.4f}")
print(f"\n6. Lowest efficiency N values (most room for improvement):")
for n, eff in sorted_eff[:10]:
    print(f"   N={n:3d}: efficiency={eff:.4f}")
print("\n7. Key insight: bbox3 optimizer is compiled and ready to run")